In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
X = torch.randn(2, 20)
net(X)

tensor([[ 0.1303,  0.1693,  0.0330,  0.0846, -0.0819, -0.0881,  0.2022, -0.2305,
         -0.3683,  0.1677],
        [-0.0254,  0.3334,  0.4270, -0.4183, -0.1438,  0.1286, -0.0079,  0.1910,
         -0.1489, -0.3123]], grad_fn=<AddmmBackward0>)

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

In [8]:
net(X)

tensor([[-0.0227,  0.1802,  0.0204, -0.1303, -0.0736, -0.2235,  0.1504,  0.2639,
         -0.0276,  0.4192],
        [-0.1514,  0.1440,  0.1279,  0.1935,  0.0037, -0.2130, -0.3043, -0.2398,
         -0.1544,  0.0863]], grad_fn=<AddmmBackward0>)

In [9]:
net2 = MySequential(net, nn.Linear(10, 1))

In [10]:
net2(X)

tensor([[-0.4191],
        [-0.2188]], grad_fn=<AddmmBackward0>)

In [11]:
net2

MySequential(
  (0): MySequential(
    (0): Linear(in_features=20, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
  (1): Linear(in_features=10, out_features=1, bias=True)
)

In [12]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()

In [13]:
net = FixedHiddenMLP()
net(X)

tensor(0.0016, grad_fn=<SumBackward0>)